# Submission Figures

In [ ]:
import sys
import os
# This makes the gdc19 package avaialble
sys.path.append('../')
import gdc19
gdc19.setup()

In [ ]:
import pandas as pd
import numpy as np
import vtki
import omfvtk
import PVGeo
import matplotlib.pyplot as plt

In [ ]:
proj = gdc19.load_project()

In [ ]:
# Add values to the meshes so colors are consistent with other data
# try:
temp_175c = proj['temp_175c']
temp_225c = proj['temp_225c']
temp_175c.point_arrays['Temperature'] = np.full(proj['temp_175c'].n_points, 175.)
temp_225c.point_arrays['Temperature'] = np.full(proj['temp_225c'].n_points, 225.)
#     tc = vtki.MultiBlock()
#     tc['temp_175c'] = proj['temp_175c']
#     tc['temp_225c'] = proj['temp_225c']
#     tc.save(gdc19.get_surfaces_path('temp-surfaces.vtm'))
# except:
#     import logging
#     logging.warning('issue.')

In [ ]:
web_friendly = False
if web_friendly:
    # do stuff
    topo = proj['land_surface']#vtki.read(gdc19.get_web_path('topo.vtk'))
    granitoid = proj['top_granitoid']#vtki.read(gdc19.get_web_path('granite.vtk'))
    texture = vtki.read_texture(gdc19.get_web_path('geology-aerial-compressed.jpg'))
    topo.textures['geo_aer'] = texture
else:
    topo = proj['land_surface']
    granitoid = proj['top_granitoid']
topo

In [ ]:
# Load kriged temperature model from Lane
# Read the kirgged model and variance
temp_grid = gdc19.load_kriged_temp()

# Remove values above topography
temp_grid = PVGeo.grids.ExtractTopography(
                remove=True, # remove the inactive cells
                tolerance=10.0 # buffer around the topo surface
               ).Apply(temp_grid, topo)
temp_grid

In [ ]:
# Add the granitoid boundary
#--
# Lets run a PVGeo filter to extract the topo surface from
#    the granitoid surface
ext = PVGeo.grids.ExtractTopography(
        remove=True, # remove the inactive cells
        tolerance=10.0 # buffer around the topo surface
        ).Apply(granitoid, topo)
granitoid = ext

In [ ]:
# Add the shape file stuff
boundary = proj['boundary']

boundary_tube = PVGeo.filters.AddCellConnToPoints(cellConn=4).Apply(boundary).tube(radius=30)


walls = PVGeo.filters.BuildSurfaceFromPoints(
                zcoords=[0., 4.5e3]).Apply(proj['boundary'])

In [ ]:
well_locs = pd.read_csv(gdc19.get_well_path('well_location_from_earth_model.csv'))
well_locs = PVGeo.pointsToPolyData(well_locs[['x', 'y', 'z (land surface)']].values).clip_box(
                    gdc19.get_roi_bounds(), invert=False)

In [ ]:
# Plot the wells
WELLS = gdc19.load_well_db()
proposed = PVGeo.filters.AddCellConnToPoints().Apply(WELLS.pop('well_new2'))#vtki.MultiBlock()
# proposed['well_new1'] = WELLS.pop('well_new1')
# proposed['well_new2'] = WELLS.pop('well_new2')

well_5832 = PVGeo.filters.AddCellConnToPoints().Apply(WELLS.pop('well_5832'))
#well_5832.set_active_scalar('ECGR')
well_Acord1 = PVGeo.filters.AddCellConnToPoints().Apply(WELLS.pop('well_Acord1'))
#well_Acord1 = WELLS.set_active_scalar('GR_SPLICE (GAPI)')

In [ ]:
proposed.plot(use_panel=True)

In [ ]:
# load the gravity model
gf = gdc19.get_gravity_path('forge_inverse_problem/RESULT_THRESHED.vtu')
grav_model = vtki.read(gf)
grav_model.active_scalar_name = 'Magnitude'
grav_model.contour()

In [ ]:
grav_model.cell_data_to_point_data().contour(6, 
                                             rng=[-0.25,0.25]
                                            ).plot(clim=0.25, 
                                                   use_panel=True)




# Helpers

Functions for adding datasets to a scene in a consistent manner

In [ ]:
POINT_SIZE = 15
LINE_WIDTH = 15
vtki.rcParams['window_size'] = np.array([1024, 768]) * 2
legend_color = vtki.parse_color('lightgrey')

In [ ]:
def clip_it(data, bounds):
    if bounds is None:
        return data
    return data.clip_box(bounds, invert=False)

def add_topo(p, bounds=None):
    data = clip_it(topo, bounds)
    if data.n_points < 1:
        return
    # We can update this to change the texture
    return p.add_mesh(data,
               texture='geo_aer', 
               name='topo', color='white')#, opacity=0.85)

def add_faults(p, bounds=None):
    data = clip_it(proj['negro_mag_fault'], bounds)
    if data.n_points < 1:
        return
    f1 = p.add_mesh(data, name='negro_mag_fault',
           show_edges=False, color='blue', opacity=0.5,
           label='Negro Mag. Fault')
    data = clip_it(proj['opal_mound_fault'], bounds)
    if data.n_points < 1:
        return
    f2 = p.add_mesh(data, name='opal_mound_fault',
           show_edges=False, color='red', opacity=0.5,
                   label='Opal Mound Fault') 
    return f1, f2

def add_granite(p, bounds=None):
    data = clip_it(granitoid, bounds)
    if data.n_points < 1:
        return
    return p.add_mesh(data, name='top_granitoid',
               show_edges=False, style='surface',
               color='grey', opacity=0.65, reset_camera=False,
               label='Top of Granite Layer')


def add_boundary(p, bounds=None):
    data = clip_it(boundary_tube, bounds)
    if data.n_points < 1:
        return
    return p.add_mesh(data, name='boundary',  color='yellow', 
                      render_lines_as_tubes=False, line_width=10,
                      label='FORGE Boundary')

def add_walls(p, bounds=None):
    data = clip_it(walls, bounds)
    if data.n_points < 1:
        return
    return p.add_mesh(data, name='walls', 
               color='yellow', opacity=0.5)


temp_d_params = dict(cmap='coolwarm', stitle='Temperature', 
                     clim=temp_grid.get_data_range('Temperature'))

def add_temp_model(p, bounds=None, contour=False, opacity=1.0):
    if contour:
        data = temp_grid.cell_data_to_point_data().contour([175, 225])
    else:
        data = temp_grid.threshold([175, 225])
    data = clip_it(data, bounds)
    if data.n_points < 1:
        return
    return p.add_mesh(data, name='temp_grid', opacity=opacity, 
                      #label='Geostatistical Temperature Model',
                      **temp_d_params)


def add_temp_probes(p, bounds=None):
    data = clip_it(proj['temperature'], bounds)
    if data.n_points < 1:
        return
    return p.add_mesh(data, name='temp', 
               point_size=POINT_SIZE, render_points_as_spheres=True,
               #label='Temperature Probes',
               **temp_d_params)

def add_temp_surfs(p, bounds=None, style='surface'):
    data = clip_it(proj['temp_175c'], bounds)
    if data.n_points < 1:
        return
    # add the temperature boundaries
    s1 = p.add_mesh(data, name='temp_175c', 
               style=style, opacity=0.7, 
#                     lighting=False,
                    **temp_d_params)
    data = clip_it(proj['temp_225c'], bounds)
    if data.n_points < 1:
        return
    s2 = p.add_mesh(data, name='temp_225c', 
               style=style, opacity=0.7, 
#                lighting=False,
                **temp_d_params)
    return s1, s2


def add_well_collars(p, bounds=None):
    data = clip_it(well_locs, bounds)
    if data.n_points < 1:
        return
    return p.add_mesh(data, name='well_locations', 
                      point_size=POINT_SIZE, color='orange',
                      label='Well Collars')

def add_well_traj(p, bounds=None):
#     data = clip_it(WELLS, bounds)
#     if data.n_points < 1:
#         return
    return p.add_mesh(WELLS, color='grey', name='WELLS', 
                      reset_camera=False, render_lines_as_tubes=False, line_width=10,
                      )

WELL_COLOR = 'mediumvioletred'
def add_well_traj_proposed(p, bounds=None):
#     data = clip_it(proposed, bounds)
#     if data.n_points < 1:
#         return
    return p.add_mesh(proposed, color=WELL_COLOR, name='proposed-wells', 
                      reset_camera=False, render_lines_as_tubes=False, line_width=LINE_WIDTH,
                     label='Proposed Well')


def add_wells_with_data(p, bounds=None):
    title = 'ECGR' # Shows up as label on scalar bar
    data = clip_it(well_5832, bounds)
    if data.n_points < 1:
        return
    w1 = p.add_mesh(data, scalars='ECGR', 
                   name='5832', clim=[0,200], 
                   cmap='viridis', stitle=title,
                   render_lines_as_tubes=False, line_width=10)
    data = clip_it(well_Acord1, bounds)
    if data.n_points < 1:
        return
    w2 = p.add_mesh(data, 
                   scalars=' GR_SPLICE (GAPI)', 
                   name = 'Acord1', 
                   clim=[0, 200], cmap = 'viridis',
                   stitle=title,
                   render_lines_as_tubes=False, line_width=10)
    return w1, w2


def add_wells_with_data_solid(p, bounds=None):
    title = 'My Title'
    data = clip_it(well_5832, bounds)
    if data.n_points < 1:
        return
    w1 = p.add_mesh(data,
                   name='5832', color='gray',
                   render_lines_as_tubes=False, line_width=10)
    data = clip_it(well_Acord1, bounds)
    if data.n_points < 1:
        return
    w2 = p.add_mesh(data, 
                   name = 'Acord1', color='gray', 
                    render_lines_as_tubes=False, line_width=10)
    return w1, w2




def add_grav_model(p, bounds=None, opacity=1.0, rng=[-0.25,0.25]):
    data = grav_model.cell_data_to_point_data().contour(6, 
                                             rng=rng)
    #data = grav_model.threshold(0.091)
    data = clip_it(data, bounds)
    if data.n_points < 1:
        return
    return p.add_mesh(data, name='grav_model', clim=rng, 
                      opacity=opacity, cmap='jet',
                    stitle='Inverted Density Model'
                      #label='Density Model', 
                        )

In [ ]:
# well_locs.save('/Volumes/GoogleDrive/My Drive/utah-forge/well_locs.vtu')
# well_locs

## Figure 1
This figure shows the a model of Milford Valley, Utah with topography, fault, granite, FORGE site and well locations and data

In [ ]:
def fig_1(p, bounds=None):
    add_topo(p, bounds)
    add_faults(p, bounds)
    add_granite(p, bounds)
    add_boundary(p, bounds)
    add_well_collars(p, bounds)
    add_well_traj(p, bounds)
    add_wells_with_data_solid(p, bounds)
    return

p = vtki.Plotter(notebook=1)
fig_1(p)
p.camera_position = [(314607.07454842806, 4234127.240330922, 12678.810422767268),
                     (337504.84888541873, 4261501.390341784, 138.79188840111613),
                     (0.22757507410880431, 0.24175047512873185, 0.9432742408331761)]
p.show_grid()

p.add_legend(bcolor=legend_color, border=True, )
cpos = p.show(auto_close=False)
# p.export_vtkjs(gdc19.EXPORT_PATH.format('figure-01'))
p.screenshot('figure-01.png')
p.close()
cpos

**Figure 1 Legend:**
- Faults (blue and yellow planes)
- Granite (dark grey)
- FORGE site (yellow outline)
- Well Locations (orange dots)
- Well Trajectories (grey)

**The well locations should be reduced a bit in size**
[(318581.0684416248, 4238878.125869998, 10502.443569695455),
 (337504.84888541873, 4261501.390341784, 138.79188840111613),
 (0.2275750741088043, 0.24175047512873182, 0.943274240833176)]

# Figure 2
This figure shows a model of Milford Valley, Utah with topography, fault, granite, FORGE site and well, well logs, and temperature locations and data

In [ ]:
def fig_2(p, bounds=None):
    fig_1(p, bounds)
    add_wells_with_data(p, bounds)
    add_temp_probes(p, bounds)
#     add_temp_surfs(p, bounds)

p = vtki.Plotter(notebook=1)
fig_2(p)
add_temp_model(p, None, False, .65)
p.camera_position = [(319034.6767280643, 4229153.193113267, 2134.2689148357804),
 (337792.27022585954, 4262182.34857588, -528.1616734381239),
 (0.03428354122002461, 0.060941228582444995, 0.9975524073753104)]
p.show_grid()
p.add_legend(bcolor=legend_color, border=True, )
cpos = p.show(auto_close=False)
# p.export_vtkjs(gdc19.EXPORT_PATH.format('figure-02'))
p.screenshot('figure-02.png')
p.close()
cpos

**Figure 2 Legend:**
- Faults (blue and yellow planes)
- Granite Layer (dark grey)
- FORGE site (yellow outline)
- Well Locations (orange dots)
- Well Trajectories (grey)


- Gamma Ray Well Logs on Acord1 and 58-32 (colored well trajectories) 
- Temperature Mesh for 175ºC (pink) and 225ºC (red)
- Temperature Probe (blue to red colored dots)

### Figure Details

- [x] The Temperature dots should be reduced a bit in size

```
p.camera_position = [(324945.55139959423, 4290999.83325066, -195.7848689208751),
                     (336909.4650625, 4261896.229565, 150.0),
                     (0.002129643806474512, 0.012755630937123225, 0.9999163757518195)]
                     
                     
p.camera_position = [(315380.5242927586, 4293235.062579815, -770.1557298219668),
                     (337616.2469577216, 4262339.693805511, 183.6717727684929),
                     (-0.013525659570214996, 0.021128554734329494, 0.9996852708267883)]
                     
# Best:                     
p.camera_position = [(319034.6767280643, 4229153.193113267, 2134.2689148357804),
 (337792.27022585954, 4262182.34857588, -528.1616734381239),
 (0.03428354122002461, 0.060941228582444995, 0.9975524073753104)]
 ```

# Figure 3
This figure shows a model of Milford Valley, Utah cropped from the north side of the FORGE site to better view the subsurface within the FORGE boundary

In [ ]:
gdc19.get_roi_bounds()

In [ ]:
ROI_BOX = [329924.98816, 344152.930125, 4252833.48213,
              4264500., 
           -5000.0, 5000.0]

def fig_3(p, bounds=None):
    fig_2(p, bounds)
    p.add_mesh(vtki.Box(gdc19.get_roi_bounds()).outline(), color='k')
    p.add_mesh(vtki.Box(ROI_BOX).outline(), color='k')


p = vtki.Plotter(notebook=1)
fig_3(p, ROI_BOX)
add_temp_model(p, None, False, .65)
p.show_grid()
p.add_legend(bcolor=legend_color, border=True, )
p.camera_position = [(326221.223619594, 4269153.062663502, -655.0668370754465), 
                     (335817.3117049677, 4261819.79840792, -190.75734756687143), 
                     (-0.02827962420855367, 0.026262494408348065, 0.9992549946045177)]
p.show(auto_close=False)
# p.export_vtkjs(gdc19.EXPORT_PATH.format('figure-03'))
p.screenshot('figure-03.png')
p.close()

**Figure 3 Legend:**
- Faults (blue and yellow planes)
- Granite Layer (dark grey)
- FORGE site (yellow outline)
- Well Locations (orange dots)
- Well Trajectories (grey)
- Gamma Ray Well Logs on Acord1 and 58-32 (colored well trajectories) 
- Temperature Mesh for 175ºC (pink) and 225ºC (red)
- Temperature Probe (blue to red colored dots)

### Figure Details

```
p.camera_position = [(326221.223619594, 4269153.062663502, -655.0668370754465),
                     (335817.3117049677, 4261819.79840792, -190.75734756687143),
                     (-0.02827962420855367, 0.026262494408348065, 0.9992549946045177)]
```

# Figure 4

This figure shows a model of Milford Valley, Utah cropped to better view the FORGE site subsurface and includes our team's proposed well location and trajectory (red).

In [ ]:
def fig_4(p, bounds=None):
    fig_3(p, bounds)
    add_well_traj_proposed(p, bounds)
    add_walls(p, bounds)
    
    
    
p = vtki.Plotter(notebook=1)
fig_4(p, ROI_BOX)
add_temp_model(p, ROI_BOX)
p.show_grid()
p.add_legend(bcolor=legend_color, border=True, )
p.camera_position = [(338420.51700107113, 4274437.238773895, 2955.1608950208406),
                     (336736.7547587104, 4262114.162538592, -288.48406559676073),
                     (-0.04936350860775744, -0.24792338675624506, 0.9675211823610093)]
p.show(auto_close=False)
# p.export_vtkjs(EXPORT_PATH.format('figure-04'))
p.screenshot('figure-04.png')
p.close()

**Figure 4 Legend:**
- Faults (blue and yellow planes)
- Granite Layer (dark grey)
- FORGE site (yellow outline)
- Well Locations (orange dots)
- Well Trajectories (grey)
- Gamma Ray well logs on Acord1 and 58-32 (colored well trajectories) 
- Temperature Mesh for 175ºC (pink) and 225ºC (red)
- Temperature Probe (blue to red colored dots)


- Proposed Well (red)

### Figure Details


Adam chose:
```
p.camera_position = [(337947.2777284753, 4274251.077437216, 3782.0155544797108),
                     (336819.79099209886, 4262069.125704545, -160.4863747958867),
                     (-0.10091855957877652, -0.2978718905088982, 0.9492564359419429)]
```

Bane chose:
```
p.camera_position = [(338420.51700107113, 4274437.238773895, 2955.1608950208406),
                     (336736.7547587104, 4262114.162538592, -288.48406559676073),
                     (-0.04936350860775744, -0.24792338675624506, 0.9675211823610093)]
```

# Figure 5

This figure shows a model of Milford Valley, Utah with topography, fault, granite, FORGE site and well, well logs, proposed well, temperature, and gravity locations and data

In [ ]:
def fig_5(p, bounds=None):
    fig_4(p, bounds)
    add_grav_model(p, bounds)
    
    
    
p = vtki.Plotter(notebook=1)
fig_5(p, ROI_BOX)
p.show_grid()
p.add_legend(bcolor=legend_color, border=True, )
p.camera_position = [(338420.51700107113, 4274437.238773895, 2955.1608950208406),
                     (336736.7547587104, 4262114.162538592, -288.48406559676073),
                     (-0.04936350860775744, -0.24792338675624506, 0.9675211823610093)]
p.show(auto_close=False)
# p.export_vtkjs(gdc19.EXPORT_PATH.format('figure-05'))
p.screenshot('figure-05.png')
p.close()

Figure 5 Data:
- Topography
- Faults (blue and yellow planes)
- Granite Layer (dark grey)
- FORGE site (yellow outline)
- Well Locations (orange dots)
- Well Trajectories (grey)
- Gamma Ray well logs on Acord1 and 58-32 (colored well trajectories) 
- Temperature Mesh for 175ºC (pink) and 225ºC (red)
- Temperature Probe (blue to red colored dots)
- Proposed Well (red)


- Gravity model across Milford Valley (colored)

In [ ]:
#We seem to have a couple figures left, so what if we add one identical
#to figure 4 but a view from the front side (granite slopes up into
#the page)

# Figure 6


In [ ]:
p = vtki.Plotter(notebook=1)
fig_1(p)
LINE_WIDTH = 500

loc = proposed.points[0]
# add_well_traj_proposed(p)
loc[-1] = 1.8e3
s = vtki.Sphere(radius=40, center=loc)
p.add_mesh(s, label='Proposed Well', color=WELL_COLOR)

p.camera_position = [(335111.21558935504, 4262955.412897479, 10111.108956611326),
                     (335111.21558935504, 4262955.412897479, 150.0),
                     (0.0, 1.0, 0.0)]
p.show_grid()

p.add_legend(bcolor=legend_color, border=True, )
cpos = p.show(auto_close=False)
# p.export_vtkjs(gdc19.EXPORT_PATH.format('figure-06'))
p.screenshot('figure-06.png')
p.close()
cpos